In [1]:
import os
import pandas as pd
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_csv_agent
from langchain.agents.agent_types import AgentType
import pandas as pd
from dotenv import load_dotenv ,find_dotenv
import openai

In [2]:
load_dotenv(find_dotenv(),override=True)

True

In [3]:
openai.api_key=os.environ["OPENAI_API_KEY"]

In [4]:
df = pd.read_csv(r'matches.csv')
df[0:10]

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan
5,335987,Jaipur,2008-04-21,SR Watson,Sawai Mansingh Stadium,0,Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,bat,Rajasthan Royals,wickets,6.0,N,NaN,Aleem Dar,RB Tiffin
6,335988,Hyderabad,2008-04-22,V Sehwag,"Rajiv Gandhi International Stadium, Uppal",0,Deccan Chargers,Delhi Daredevils,Deccan Chargers,bat,Delhi Daredevils,wickets,9.0,N,NaN,IL Howell,AM Saheba
7,335989,Chennai,2008-04-23,ML Hayden,"MA Chidambaram Stadium, Chepauk",0,Chennai Super Kings,Mumbai Indians,Mumbai Indians,field,Chennai Super Kings,runs,6.0,N,NaN,DJ Harper,GA Pratapkumar
8,335990,Hyderabad,2008-04-24,YK Pathan,"Rajiv Gandhi International Stadium, Uppal",0,Deccan Chargers,Rajasthan Royals,Rajasthan Royals,field,Rajasthan Royals,wickets,3.0,N,NaN,Asad Rauf,MR Benson
9,335991,Chandigarh,2008-04-25,KC Sangakkara,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Mumbai Indians,Mumbai Indians,field,Kings XI Punjab,runs,66.0,N,NaN,Aleem Dar,AM Saheba


In [5]:
pd_agent = create_pandas_dataframe_agent(OpenAI(temperature=0), 
                         df, 
                         verbose=True)

In [6]:
pd_agent.run("Total Number of mathes played by Kolkata Knight Riders at Bangalore city")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to get the matches played by Kolkata Knight Riders at Bangalore city
Action: python_repl_ast
Action Input: df[(df['team1'] == 'Kolkata Knight Riders') & (df['city'] == 'Bangalore') | (df['team2'] == 'Kolkata Knight Riders') & (df['city'] == 'Bangalore')].shape[0]
Observation: 11
Thought: I now know the final answer
Final Answer: 11

> Finished chain.


'11'

In [7]:
csv_agent = create_csv_agent(ChatOpenAI(temperature=0,
   model="gpt-3.5-turbo-0613"), 
  'matches.csv',
   verbose=True,
   agent_type=AgentType.OPENAI_FUNCTIONS,)

In [8]:
csv_agent.run("Total Number of toss win by Kolkata Knight Riders againest Royal Challengers Bangalore")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['team1']=='Kolkata Knight Riders') & (df['team2']=='Royal Challengers Bangalore')]['toss_winner'].value_counts().sum()"}`


12The Kolkata Knight Riders have won the toss 12 times against the Royal Challengers Bangalore.

> Finished chain.


'The Kolkata Knight Riders have won the toss 12 times against the Royal Challengers Bangalore.'

In [9]:
multi_agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    ["matches.csv", "deliveries.csv"],
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [10]:
multi_agent.run('Are there any common rows in both datasets can just tell me count?')



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "common_rows = df1.merge(df2, on=['id', 'city', 'date', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner'])\ncommon_rows.shape[0]"}`


KeyError: 'city'
Invoking: `python_repl_ast` with `{'query': "common_rows = df1.merge(df2, on=['id', 'date', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner'])\ncommon_rows.shape[0]"}`


KeyError: 'date'

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.c


Invoking: `python_repl_ast` with `{'query': "common_rows = df1.merge(df2, on=['id', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner'])\ncommon_rows.shape[0]"}`


KeyError: 'team1'

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.c


Invoking: `python_repl_ast` with `{'query': "common_rows = df1.merge(df2, on=['id', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner'])\ncommon_rows.shape[0]"}`


KeyError: 'team1'

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.c


Invoking: `python_repl_ast` with `{'query': "common_rows = df1.merge(df2, on=['id'])\ncommon_rows.shape[0]"}`


193468

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0613 in organization org-C1AocB79vWA3d0mt9m7qEeAz on requests per min. Limit: 3 / min. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.c

There are 193,468 common rows in both datasets.

> Finished chain.


'There are 193,468 common rows in both datasets.'